# Laboratorio: Implementación de un modelo de Machine Learning

Esta experiencia de laboratorio se trata de implementar un modelo de ML para clasificación binaria usando datos tabulares. Para eso usaremos el modelo ya ajustado en el notebook [00_supervivencia_titanic.ipynb]. Para implementar el modelo como un servicio usaremos la popular biblioteca [`fastAPI`](https://fastapi.tiangolo.com/).

## Clasificación binaria de datos tabulares usando Regresión Logística

### Creando la función predict_supervivencia_titanic

Vamos a crear el método `predict_supervivencia_titanic` que toma como entradas un **array con los valores de las características del viaje** y un **umbral de confianza**. La función determinará si el viaje es de clase propina alta o propina baja, devolviendo un 1 o un 0 dependiendo del caso.

La salida del modelo es un vector de probabilidades de pertenencia del viaje a alguna de las dos clases posibles. El último argumento de entrada a nuestra función (el nivel de confianza) será el umbral que dichas probabilidades deben superar para determinar que el viaje en cuestión si representa uno de propina alta. Por defecto `predict_taxi_trip` usa el valor 0.5 para esto.

In [37]:
# === 01_server_titanic.ipynb ===
# Servidor FastAPI para el modelo de supervivencia del Titanic

import os
import joblib
import pandas as pd
import numpy as np

from typing import Optional, Literal

import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field

In [39]:
# -----------------------------
# 1) Cargar artefacto del modelo
# -----------------------------
ARTIFACT_PATH = "model/logistic_titanic_pipeline.pkl"
if not os.path.exists(ARTIFACT_PATH):
    raise FileNotFoundError(
        f"No se encontró {ARTIFACT_PATH}. Entrena y guarda el modelo en el notebook 00 primero."
    )

artifact = joblib.load(ARTIFACT_PATH)
pipe = artifact["model"]
DEFAULT_THRESHOLD = float(artifact.get("threshold", 0.5))
EXPECTED_FEATURES = artifact.get(
    "features",
    ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']  # fallback
)

In [41]:
# -----------------------------
# 2) Definición de la API
# -----------------------------
app = FastAPI(
    title="API - Supervivencia Titanic",
    description="Clasificador binario de supervivencia usando Pipeline de scikit-learn.",
    version="1.0.0"
)

# Schema de entrada (campos crudos; el Pipeline se encarga del preprocesamiento)
class Passenger(BaseModel):
    pclass: Literal[1, 2, 3] = Field(..., description="Clase del boleto (1, 2, 3)")
    sex: Literal["male", "female"] = Field(..., description="Sexo del pasajero")
    age: Optional[float] = Field(None, ge=0, le=100, description="Edad en años (puede ser nula)")
    sibsp: int = Field(..., ge=0, description="Hnos/cónyuges a bordo")
    parch: int = Field(..., ge=0, description="Padres/hijos a bordo")
    fare: Optional[float] = Field(None, ge=0, description="Tarifa pagada (puede ser nula)")
    embarked: Optional[Literal["C", "Q", "S"]] = Field(None, description="Puerto de embarque")

    # normalizamos por si vienen en minúsculas / mayúsculas mezcladas
    def to_dataframe(self) -> pd.DataFrame:
        row = {
            "pclass": int(self.pclass),
            "sex": str(self.sex).lower(),
            "age": self.age,
            "sibsp": int(self.sibsp),
            "parch": int(self.parch),
            "fare": self.fare,
            "embarked": None if self.embarked is None else str(self.embarked).upper(),
        }
        # asegurar orden de columnas esperado por el pipeline
        return pd.DataFrame([row], columns=EXPECTED_FEATURES)


@app.get("/")
def home():
    return {
        "message": "¡API Titanic OK! Visita /docs para probar.",
        "default_threshold": DEFAULT_THRESHOLD,
        "expected_features": EXPECTED_FEATURES,
    }


@app.get("/healthz")
def healthz():
    return {"status": "healthy"}


@app.post("/predict")
def predict(passenger: Passenger, confidence: Optional[float] = None):
    """
    Predice supervivencia para un pasajero.
    - Usa 'confidence' como umbral si se especifica; de lo contrario usa el umbral guardado.
    """
    try:
        X = passenger.to_dataframe()
        # Probabilidad de clase positiva (sobrevive = 1)
        proba = float(pipe.predict_proba(X)[0, 1])
        thr = float(confidence) if confidence is not None else DEFAULT_THRESHOLD
        pred = int(proba >= thr)
        label = "Sobrevive" if pred == 1 else "No sobrevive"
        return {
            "threshold_used": thr,
            "prob_survive": proba,
            "pred_class": int(pred),
            "pred_label": label,
            "features_order": EXPECTED_FEATURES,
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error en predicción: {e}")

In [ ]:
# -----------------------------
# 3) Ejecutar servidor (en notebook)
# -----------------------------

nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [14236]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:49394 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:49395 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:49396 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:49397 - "POST /predict?confidence=0.55 HTTP/1.1" 200 OK
INFO:     127.0.0.1:49398 - "GET /predict?confidence=0.55 HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:49398 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:49403 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:49402 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:49402 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:49405 - "POST /predict HTTP/1.1" 200 OK
